<a href="https://colab.research.google.com/github/HUMINTING/callte-detetion-detail/blob/20230105/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
from google.colab import drive
drive.mount('/content/drive')
sys.path.append('/content/drive/My Drive/mt_detection/20230124')

Mounted at /content/drive


In [ ]:
pip install pretrainedmodels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60966 sha256=d5cbc4cbd56d952e7a3d5c7d27a825cde8ebae562db0705fcaa80119f53478ee
  Stored in directory: /root/.cache/pip/wheels/ed/fa/b9/5c82b59d905f95542a192b883c0cc0082407ea2f54beb2f9e6
Successfully built pretrainedmodels


In [ ]:
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from my_nn import MyNetBase, MyNetMultiTask, MyNetMultiTask_change_model, MyNetMultiTask_devide_hip_head, MyNetMultiTask_devide_hip_direction, MyNetMultiTask_devide_hip_direction_hierarchical


In [ ]:
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models
#from effnetv2 import effnetv2_s
from torch.autograd import Variable

 
# 设置超参数
 
BATCH_SIZE = 32
EPOCHS = 100
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
 
# 数据预处理
 
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    # transforms.RandomVerticalFlip(),
    # transforms.RandomCrop(50),
    # transforms.ColorJitter(brightness=0.5, contrast=0.5, hue=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
 
])
transform_test = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
# 读取数据
dataset_train = datasets.ImageFolder('/content/drive/MyDrive/mt_detection/20230124/train', transform)
print(dataset_train.imgs)
# 对应文件夹的label
print(dataset_train.class_to_idx)
dataset_test = datasets.ImageFolder('/content/drive/MyDrive/mt_detection/20230124/valid', transform_test)
# 对应文件夹的label
print(dataset_test.class_to_idx)
 
# 导入数据
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=False)
modellr = 1e-4
 
# 实例化模型并且移动到GPU
criterion = nn.CrossEntropyLoss()
# model = effnetv2_s()
# num_ftrs = model.classifier.in_features
# model.classifier = nn.Linear(num_ftrs, 2)
model = MyNetBase("densenet")
model.to(DEVICE)
# 选择简单暴力的Adam优化器，学习率调低
optimizer = optim.Adam(model.parameters(), lr=modellr)
#optimizer = optim.SGD(model.parameters(), lr=modellr)


def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    modellrnew = modellr * (0.1 ** (epoch // 50))
    print("lr:", modellrnew)
    for param_group in optimizer.param_groups:
        param_group['lr'] = modellrnew


 
# 定义训练过程
 
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    sum_loss = 0
    correct = 0
    total_num = len(train_loader.dataset)
    print(total_num, len(train_loader))
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).to(device), Variable(target).to(device)
        output = model(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print_loss = loss.data.item()
        sum_loss += print_loss
        _, pred = torch.max(output.data, 1)
        correct += torch.sum(pred == target)
        acc = correct / total_num
        if (batch_idx + 1) % 50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (batch_idx + 1) * len(data), len(train_loader.dataset),
                       100. * (batch_idx + 1) / len(train_loader), loss.item()))
    ave_loss = sum_loss / len(train_loader)
    print('epoch:{},loss:{},Accuracy: {}/{} ({:.0f}%)\n'.format(epoch, ave_loss, correct, len(test_loader.dataset), 100 * acc))
 

def val(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    total_num = len(test_loader.dataset)
    print(total_num, len(test_loader))
    with torch.no_grad():
        for data, target in test_loader:
            data, target = Variable(data).to(device), Variable(target).to(device)
            output = model(data)
            loss = criterion(output, target)
            _, pred = torch.max(output.data, 1)
            correct += torch.sum(pred == target)
            print_loss = loss.data.item()
            test_loss += print_loss
        correct = correct.data.item()
        acc = correct / total_num
        avgloss = test_loss / len(test_loader)
        print('\nVal set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            avgloss, correct, len(test_loader.dataset), 100 * acc))
 

 
# 训练
 
for epoch in range(1, EPOCHS + 1):
    adjust_learning_rate(optimizer, epoch)
    train(model, DEVICE, train_loader, optimizer, epoch)
    #val(model, DEVICE, test_loader)
torch.save(model.state_dict(),'/content/drive/MyDrive/mt_detection/20230124/weights/model_densenet161/epoch_100.pth' )

[('/content/drive/MyDrive/mt_detection/20230124/train/garbage/estrus_data_one_1_20160706_1800_ch1_0208_02.png', 0), ('/content/drive/MyDrive/mt_detection/20230124/train/garbage/estrus_data_one_1_20160706_1800_ch1_0444_03.png', 0), ('/content/drive/MyDrive/mt_detection/20230124/train/garbage/estrus_data_one_1_20160706_1800_ch1_0445_03.png', 0), ('/content/drive/MyDrive/mt_detection/20230124/train/garbage/estrus_data_one_1_20160706_1800_ch1_0447_04.png', 0), ('/content/drive/MyDrive/mt_detection/20230124/train/garbage/estrus_data_one_1_20160706_1800_ch1_0769_06.png', 0), ('/content/drive/MyDrive/mt_detection/20230124/train/garbage/estrus_data_one_1_20160706_1800_ch1_0772_06.png', 0), ('/content/drive/MyDrive/mt_detection/20230124/train/garbage/estrus_data_one_1_20160706_1800_ch1_0773_06.png', 0), ('/content/drive/MyDrive/mt_detection/20230124/train/garbage/estrus_data_one_1_20160706_1800_ch1_0773_07.png', 0), ('/content/drive/MyDrive/mt_detection/20230124/train/garbage/estrus_data_one_1_

# 数据预处理